# __Regressions & Plotting__

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
#Obsolete at the moment
# Price_Patent_Reg = pd.read_csv('data/Price_Patent_Reg.csv', index_col = 0, dtype = data_types)

In [1]:
import dill
Price_Patent_Reg = dill.load(open('data/features_created.pkd', 'rb'))

### __Drop unneeded columns__
Dropping several columns that have low amounts of data (you have to go back to previous notebooks to see this is the case), or have a weak theoretical basis for being included.

In [3]:
Price_Patent_Reg = Price_Patent_Reg.drop([
                                         'classification_for_rate_setting',
                                         'corresponding_generic_drug_nadac_per_unit',
                                         'pricing_unit',
                                         'ingredient',
                                         'applicant',
                                         'otc',
                                         'type',
                                         'dosage_form',
                                         'route',
                                         ], axis = 1)
# Price_Patent_Reg.info()

### __Create test & train datasets__
From the dependent and independent variable dataframes

In [6]:
Price_Patent_Reg = pd.get_dummies(Price_Patent_Reg, drop_first = True)
Price_Patent_Reg.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214082 entries, 68472 to 6411864
Data columns (total 12 columns):
nadac_per_unit                                214082 non-null float32
ndc                                           214082 non-null float32
days_before_patent_expires                    214082 non-null float32
drug_age                                      214082 non-null float32
generic_exists                                214082 non-null int32
effective_date_year                           214082 non-null float16
effective_date_month                          214082 non-null float16
effective_date_day                            214082 non-null float16
corresponding_generic_drug_effective_year     214082 non-null float16
corresponding_generic_drug_effective_month    214082 non-null float16
corresponding_generic_drug_effective_day      214082 non-null float16
te_code_A                                     214082 non-null uint8
dtypes: float16(6), float32(4), int32(1), uint8(

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(Price_Patent_Reg,
                                         test_size = 0.2,
                                         random_state = 1,
#                                          shuffle = True
                                        )    #shuffle data to avoid correlation to the natural order of the data

In [8]:
test_data.head()

,nadac_per_unit,ndc,days_before_patent_expires,drug_age,generic_exists,effective_date_year,effective_date_month,effective_date_day,corresponding_generic_drug_effective_year,corresponding_generic_drug_effective_month,corresponding_generic_drug_effective_day,te_code_A
2509598,0.05078,2.930001e+10,0.0,2140.0,0,2017.0,6.0,27.0,0.0,0.0,0.0,1
3532416,0.03889,5.107904e+10,0.0,268.0,0,2014.0,1.0,11.0,0.0,0.0,0.0,0
1726471,2.16548,7.815936e+08,0.0,2122.0,0,2014.0,12.0,27.0,0.0,0.0,0.0,0
438940,0.10550,9.394775e+07,0.0,1271.0,0,2018.0,10.0,16.0,0.0,0.0,0.0,1
5912535,1.01817,6.808406e+10,0.0,1954.0,0,2014.0,10.0,23.0,0.0,0.0,0.0,1


### __Create a class for estimating by group__

In [9]:
from sklearn import base
import numpy as np 
import pandas as pd

class GroupbyEstimator(base.BaseEstimator, base.RegressorMixin):

    
    def __init__(self, groupby_column, pipeline_factory):
        # column is the value to group by; estimator_factory can be called to produce estimators
        self.groupby_column = groupby_column
        self.pipeline_factory = pipeline_factory

    
    def fit(self, dataframe, label):
        # Create an estimator and fit it with the portion in each group (create and fit a model per city
        self.drugs_dict = {}
        self.label = label
        self.coefs_dict = {} 
        self.intercepts_dict = {} 
       
        dataframe = pd.get_dummies(dataframe)  #onehot encoder had problems with the data, so I'm getting the dummies with pandas here
        
        for name, values in dataframe.groupby(self.groupby_column):
            y = values[label]
            X = values.drop(columns = [label, self.groupby_column], axis = 1)
            self.drugs_dict[name] = self.pipeline_factory().fit(X, y)
            self.coefs_dict[name] = self.drugs_dict[name].named_steps["lin_reg"].coef_
            self.intercepts_dict[name] = self.drugs_dict[name].named_steps["lin_reg"].intercept_
        return self

    #Method to get the coefficients for each regression
    def get_coefs(self):       
        return self.coefs_dict
    
    #Method to get the intercepts for each regression
    def get_intercepts(self):
        return self.intercepts_dict
    
        
    def predict(self, test_data):
        price_pred_list = []

        for idx, row in test_data.iterrows():
            name = row[self.groupby_column]                                 #get drug name from drug column
            regression_coefs = self.drugs_dict[name]                        #get coefficients from fitting in drugs_dict
            row = pd.DataFrame(row).T
            X = row.drop(columns = [self.label, self.groupby_column], axis = 1).values.reshape(1, -1)            

            drug_price_pred = regression_coefs.predict(X)    
            price_pred_list.append([name, drug_price_pred])
        return price_pred_list

### __Run Linear Regression__

In [10]:
from sklearn import base
import numpy as np 

def pipeline_factory():
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LinearRegression

    return Pipeline([
                     ('lin_reg', LinearRegression())
                    ])

lin_model = GroupbyEstimator('ndc', pipeline_factory).fit(train_data,'nadac_per_unit')

In [11]:
# Predict & save results
results = lin_model.predict(test_data)
predictions = [x[1][0] for x in results]
actual = test_data.iloc[:,0]

### __Calculating scoring metrics__

And an explanation of each score metric being considered

In [12]:
from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 

scoring_methods = [
                   explained_variance_score, # 1-(Var(predicted-true)/Var(true)); equal to R2 if mean(error) == 0 (e.g. true == 0)
                   max_error,                # captures the worst case error(residual) between the predicted value and the true value
                   mean_absolute_error,      # average of (the absolute value of) all residuals; less sensitive to outliers; lower is better
                   mean_squared_error,       # penalty for making more predictions varying from the actual value; more sensitive to outliers
                   mean_squared_log_error,   # treats small differences between small true and predicted differences the same as big differences between large true and predicted values
                   median_absolute_error,    # Robust (insensitive) to outliers
                   r2_score                  # The proportion of variance of the dependent variable that has been explained by the independent variables (multioutput param defaults to 'uniform_average')
                  ]

for method in scoring_methods:
    try: 
        score = method(actual, predictions)
        print(method, ': ', score)
    except ValueError:
        pass

<function explained_variance_score at 0x000001D180DDD7B8> :  0.9923553609463692
<function max_error at 0x000001D180DDD8C8> :  6.771544137277942
<function mean_absolute_error at 0x000001D180DDD598> :  0.3299878600049433
<function mean_squared_error at 0x000001D180DDD620> :  0.5012352619859909
<function median_absolute_error at 0x000001D180DDD730> :  0.11154743341376161
<function r2_score at 0x000001D180DDD840> :  0.9923552650177279


### __R2s Scores of Various Random States__ 

This was done to confirm no problem with data shuffling.

* seed 7: 0.9922258723779824
* seed 6: 0.9922144284909121
* seed 5: 0.9922179390324788
* seed 4: 0.9923285055598297
* seed 3: 0.9923976517249262
* seed 2: 0.9921732472740931
* seed 1: 0.9923552650177279

In [14]:
# Prediction/Actual values in dataframe for comparison (index is NDC number)
prediction_values = pd.DataFrame({'actual':actual, 'predictions': predictions})
prediction_values

,actual,predictions
2509598,0.05078,0.048565
3532416,0.03889,0.457530
1726471,2.16548,2.258173
438940,0.10550,0.093438
5912535,1.01817,1.038753
72085,2.70332,3.412836
174029,1.20522,1.174578
6405554,10.13076,8.650459
5292429,0.05644,-0.005692
4384145,3.30011,3.128525


### __Retrieve coefficients & intercepts for modeling__

In [54]:
# Retrieve all model coefficients 
all_model_coefs = lin_model.get_coefs()

# Retrieve all model intercepts 
all_model_intercepts = lin_model.get_intercepts()

#Convert coefficients to dataframe
col_names = train_data.iloc[:,1:].columns
fit_details = pd.DataFrame(all_model_coefs).transpose()
fit_details.reset_index(inplace = True)
fit_details.columns = col_names

#Incorporate intercepts to dataframe
intercepts = pd.DataFrame(all_model_intercepts, index = ['intercepts']).transpose()
fit_details = fit_details.merge(intercepts, left_on = 'ndc', right_on = intercepts.index)

# Send final data to CSV
fit_details.to_csv('data/fit_details.csv')
# fit_details.head()

### __Prep data for plotting__

In [191]:
# Prep data for plotting
plotting_data = Price_Patent_Reg[['nadac_per_unit', 'ndc', 'effective_date_day', 'effective_date_month', 'effective_date_year']] #select columns
plotting_data = plotting_data.rename(columns = {'effective_date_day' : 'day', #change column names so that they're readable by to_datetime method
                                               'effective_date_month' : 'month',
                                               'effective_date_year' : 'year'},
                                               )
#change columns to datetime
plotting_data['date'] = pd.to_datetime(plotting_data[['year', 'month', 'day']].astype(int).astype(str).agg('-'.join, axis = 1))

plotting_data['ndc'] = plotting_data['ndc'].astype('int') #offending line...reduces ndc count

len(plotting_data.ndc.unique().tolist())

plotting_data = plotting_data.drop(['year', 'month', 'day'], axis = 1) #drop piecemeal date columns
plotting_data = plotting_data[['nadac_per_unit', 'date', 'ndc']] #reorder columns
# Send final data to CSV
plotting_data.to_csv('data/plotting_data.csv')

## __Plot Drug Prices and Forecasts__

* Note that the following plot is still in development.  
* The current _working_ version (`Drug_Price_Plots.py` can be run from the command line via `bokeh serve --show Drug_Price_Plots.py`

In [198]:
import pandas as pd
from bokeh.io import curdoc, output_notebook, output_file
from bokeh.layouts import row, column
from bokeh.models import Select, DataRange1d, ColumnDataSource
from bokeh.plotting import figure
import dateutil.parser as dateparser

#Must be run from the command line

output_notebook()
output_file('test.html')

def get_dataset(src, drug_id):
    src.drop('Unnamed: 0', axis = 1, inplace = True)
    df = src.loc[src['ndc'] == int(drug_id)]
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index(['date'])
    df.sort_index(inplace=True)
    source1 = ColumnDataSource(data=df)
    return source

def get_predictions(src, drug_id):
    #Get input data (idealling coming from calendar button)
    input_date = datetime('2020-03-01') #make this selectable in future
    #Get current ndc
    ver = ver_select.value
    #Split it up
    input_coefs = {'days_before_patent_expires': 0,
                   'drug_age' : 0,
                   'generic_exists' : 0,
                   'effective_date_year' : input_date.year, #user input
                   'effective_date_month' : input_date.month, #user input
                   'effective_date_day' : input_date.day, #user input
                   'corresponding_generic_drug_effective_year' : 0,
                   'corresponding_generic_drug_effective_month' : 0,
                   'corresponding_generic_drug_effective_day' : 0,
                   'te_code_A' : te_code, #user input
                   'intercepts' : 0}
    df2 = fit_details.loc[fit_details['ndc'] == int(drug_id)]
    df2 = df2 * input_coefs
    
def make_plot(source, title):
    #Historical Data
    plot = figure(plot_width=800, plot_height = 800, tools="", x_axis_type = 'datetime')#, toolbar_location=None)
    plot.xaxis.axis_label = 'Time'
    plot.yaxis.axis_label = 'Price ($)'
    plot.axis.axis_label_text_font_style = 'bold'
    plot.x_range = DataRange1d(range_padding = 0.0)
    plot.grid.grid_line_alpha = 0.3 
    plot.title.text = title
    plot.line(x= 'date', y='nadac_per_unit', source=source)
    return plot


def update_plot(attrname, old, new):
    ver = vselect.value
    df1 = df.loc[df['ndc'] == int(new)]
    df1['date'] = pd.to_datetime(df1['date'])
    df1 = df1.set_index(['date'])
    df1.sort_index(inplace=True)
    newSource = ColumnDataSource(df1) 
    source1.data = newSource.data

def update_prediction(attrname, old, new):
    ver = vselect.value
    input_date = dateparser.parse('2013-03-01').date()
    input_coefs = {'days_before_patent_expires': 0,
                   'drug_age' : 0,
                   'generic_exists' : 0,
                   'effective_date_year' : input_date.year, #user input
                   'effective_date_month' : input_date.month, #user input
                   'effective_date_day' : input_date.day, #user input
                   'corresponding_generic_drug_effective_year' : 0,
                   'corresponding_generic_drug_effective_month' : 0,
                   'corresponding_generic_drug_effective_day' : 0,
                   'te_code_A' : te_code, #user input
                   'intercepts' : 0}
    df2 = pd.DataFrame(input_coefs) #convert input data to dataframe format
    results = lin_model.predict(df2)#get predictions via groupestimator class --> need to bring lin_reg into this cell?
    #Create datetime
    
    df2 = df2.set_index(['date'])
    df2.sort_index(inplace = True)
    newSource = ColumnDataSource(df2)
    source2.data = newSource.data
    
df = pd.read_csv('data/plotting_data.csv')
fit_details = pd.read_csv('data/fit_details.csv')
ver = '54034808' #Initial id number
cc = df['ndc'].astype(str).unique() #select-menu options

#Select (dropdown) menu
vselect = Select(value=ver, title='Drug ID', options=sorted((cc)))

source1 = get_dataset(df, ver)
source2 = get_dataset(fit_details, ver)
plot = make_plot(source, "Drug Prices")

vselect.on_change('value', update_plot)
controls = row(vselect)

curdoc().add_root(row(plot, controls))

Loading BokehJS ...

C:\Users\The Fat Lady\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
